This Python wrapper for the ORCID Public API v1.2 has been developed for the SHARE Curation Associates group working to integrate selected ORCID data into SHARE Notify.  It presupposes that Python 3, Jupyter Notebook, Requests and JSON dependencies have been met.
Understand that only ORCID profiles in the public (production) server with visibility set to "public" can be captured by these queries.

In [80]:
Public_ORCID_API_BASE_URI = 'https://pub.orcid.org/v1.2/'

A known ORCID ID:

In [81]:
ORCID_ID = '0000-0002-7139-2625/'

Possible API types, not including Lucene, for public API v1.2: 


orcid-profile/ [returns entire ORCID] 
orcid-bio/ [returns names, biography, emails, external identifiers, keywords, researcher IDs] 
orcid-works/ 
affiliations/ [returns educational history and employment] 
funding/

In [82]:
API_TYPE = 'orcid-bio/'
URI = Public_ORCID_API_BASE_URI + ORCID_ID + API_TYPE

Unparsed JSON results:

In [83]:
import requests
import json

payload={'Content-Type': 'application/orcid+json'}

r = requests.get(URI, json=payload)
print(r.url)
r.json()

https://pub.orcid.org/v1.2/0000-0002-7139-2625/orcid-bio/


{'error-desc': None,
 'message-version': '1.2',
 'orcid-profile': {'client-type': None,
  'group-type': None,
  'orcid': None,
  'orcid-activities': None,
  'orcid-bio': {'biography': None,
   'contact-details': {'address': {'country': {'value': 'US',
      'visibility': 'PUBLIC'}},
    'email': []},
   'delegation': None,
   'external-identifiers': {'external-identifier': [{'external-id-common-name': {'value': 'ResearcherID'},
      'external-id-orcid': None,
      'external-id-reference': {'value': 'O-1907-2013'},
      'external-id-source': None,
      'external-id-url': {'value': 'http://www.researcherid.com/rid/O-1907-2013'},
      'orcid': None,
      'source': {'source-client-id': None,
       'source-date': {'value': 1387901349957},
       'source-name': None,
       'source-orcid': {'host': 'orcid.org',
        'path': '0000-0001-7707-4137',
        'uri': 'http://orcid.org/0000-0001-7707-4137',
        'value': None}}},
     {'external-id-common-name': {'value': 'ISNI'},
    

Let's parse out the external (researcher) identifiers [limited by the ORCID v1.2 schema to two] and the other external researcher IDs [no max]:

In [84]:
r = requests.get(URI, json=payload).json()
print('External identifiers:')
results = r['orcid-profile']['orcid-bio']['external-identifiers']['external-identifier']
for i in results:
    print(i['external-id-url']['value'])
    

print ('Researcher identifiers:')
results2 = r['orcid-profile']['orcid-bio']['researcher-urls']['researcher-url']
for i in results2:
    print(i['url']['value'])

External identifiers:
http://www.researcherid.com/rid/O-1907-2013
http://isni.org/isni/0000000108836425
Researcher identifiers:
http://isni.org/isni/0000000108836425
http://lccn.loc.gov/no96037730
http://scholar.google.com/citations?user=9YUrD_8AAAAJ&hl=en
http://viaf.org/viaf/271237784/#Holloway,_Steven_W._(Steven_Winford),_1955-
http://www.linkedin.com/pub/steven-holloway/22/46a/b86/
http://www.researcherid.com/rid/O-1907-2013
http://www.researchgate.net/profile/Steven_Holloway2
http://www.worldcat.org/wcidentities/lccn-no96-37730
https://jmu.academia.edu/StevenHolloway
https://works.bepress.com/steven_holloway/
https://www.lib.jmu.edu/staffdir/bio.aspx?uid=293


A Lucene search uses the same ORCID API query types above, with the expected LUCENE syntax.  We'll create a modified URI, and write a query for ORCID holders with the family name "Holloway":

In [69]:
QUERY_TYPE = 'orcid-bio/'
URI = Public_ORCID_API + ORCID_ID + QUERY_TYPE

In [85]:
Q_URI = "https://pub.orcid.org/v1.2/search/" + API_TYPE + '?q='

The specific Lucene query:

In [86]:
LUCENE = 'family-name:Holloway'
L_QUERY = Q_URI + LUCENE + '&start=0&row=10'

In [ ]:
The unparsed results:

In [87]:
r = requests.get(L_QUERY, json=payload)
print(r.url)
r.json()

https://pub.orcid.org/v1.2/search/orcid-bio/?q=family-name:Holloway&start=0&row=10


{'error-desc': None,
 'message-version': '1.2',
 'orcid-profile': None,
 'orcid-search-results': {'num-found': 101,
  'orcid-search-result': [{'orcid-profile': {'client-type': None,
     'group-type': None,
     'orcid': None,
     'orcid-activities': None,
     'orcid-bio': {'biography': None,
      'contact-details': None,
      'delegation': None,
      'external-identifiers': {'external-identifier': [{'external-id-common-name': {'value': 'ResearcherID'},
         'external-id-orcid': None,
         'external-id-reference': {'value': 'F-9911-2012'},
         'external-id-source': None,
         'external-id-url': {'value': 'http://www.researcherid.com/rid/F-9911-2012'},
         'orcid': None,
         'source': {'source-client-id': None,
          'source-date': {'value': 1374076989961},
          'source-name': None,
          'source-orcid': {'host': 'orcid.org',
           'path': '0000-0001-7707-4137',
           'uri': 'http://orcid.org/0000-0001-7707-4137',
           'value'

Let's parse out the ORCID IDs for the Holloways in the production server.  The API throttles the number of hits displayed per query to 10:

In [88]:
r = requests.get(L_QUERY, json=payload).json()

hits = r['orcid-search-results']['num-found']
print('Total number of hits:', hits)


print ('ORCID-IDs for first ten Holloways:')
results = r['orcid-search-results']['orcid-search-result']

for i in results:
    print(i['orcid-profile']['orcid-identifier']['uri'])

Total number of hits: 101
ORCID-IDs for first ten Holloways:
http://orcid.org/0000-0002-4585-9594
http://orcid.org/0000-0001-9270-6198
http://orcid.org/0000-0002-3611-4726
http://orcid.org/0000-0001-9527-794X
http://orcid.org/0000-0003-0230-4991
http://orcid.org/0000-0002-1947-4885
http://orcid.org/0000-0002-7662-6638
http://orcid.org/0000-0001-8900-5315
http://orcid.org/0000-0001-8085-1128
http://orcid.org/0000-0003-3089-2006


Wanna search for ORCID holders from your school?  The ORCID spec uses unique 4-digit RINGGOLD values for registered institutions -- easy, right?  The API does not directly support it -- go figure.  Your best bet is a LUCENE search with the RINGGOLD ID AND the school name, querying the orcid-bio section. So, for instance, in order to search for James Madison University ORCID holders, I use

In [89]:
L_QUERY = 'https://pub.orcid.org/v1.2/search/orcid-bio/?q=3745+AND+"James+Madison+University"&start=0&row=100'

r = requests.get(L_QUERY, json=payload)
print(r.url)
r.json()

https://pub.orcid.org/v1.2/search/orcid-bio/?q=3745+AND+%22James+Madison+University%22&start=0&row=100


{'error-desc': None,
 'message-version': '1.2',
 'orcid-profile': None,
 'orcid-search-results': {'num-found': 132,
  'orcid-search-result': [{'orcid-profile': {'client-type': None,
     'group-type': None,
     'orcid': None,
     'orcid-activities': None,
     'orcid-bio': {'biography': None,
      'contact-details': None,
      'delegation': None,
      'external-identifiers': None,
      'keywords': None,
      'personal-details': {'credit-name': None,
       'family-name': {'value': 'Rogers', 'visibility': 'PUBLIC'},
       'given-names': {'value': 'Elizabeth', 'visibility': 'PUBLIC'},
       'other-names': None},
      'researcher-urls': None,
      'scope': None},
     'orcid-deprecated': None,
     'orcid-history': None,
     'orcid-id': None,
     'orcid-identifier': {'host': 'orcid.org',
      'path': '0000-0003-3171-8579',
      'uri': 'http://orcid.org/0000-0003-3171-8579',
      'value': None},
     'orcid-internal': None,
     'orcid-preferences': None,
     'type': None}

Finally, let's parse out some bibliographic information from a single ORCID profile.  Warned you --- it's MESSY.

In [90]:
PUB_URI = 'https://pub.orcid.org/v1.2/0000-0002-7139-2625/orcid-works/'

r = requests.get(PUB_URI, json=payload).json()

print ('Works:') 
results = r['orcid-profile']['orcid-activities']['orcid-works']['orcid-work']


for i in results:
    print('========')
    print('Publication titles:')
    print(i['work-title']['title']['value'])
    print('Work type:') 
    print(i['work-type'])

#work-citations may be any mix of BIBTEX and other formatted text, too 
#complicated for this simple python loop to handle, but you get the idea...

for i in results:
    print('========')
    print('Citation:')
    print(i['work-citation']['citation'])
    print('Citation type:')
    print(i['work-citation']['work-citation-type'])
    print('Work type:') 
    print(i['work-type'])

Works:
Publication titles:
Rainer Albertz, A History of Israelite Religion in the Old Testament Period, volume 1: From the Beginning to the End of the Monarchy
Work type:
BOOK_REVIEW
Publication titles:
Use of Assyriology in Chronological Apologetics in David’s Secret Demons
Work type:
JOURNAL_ARTICLE
Publication titles:
Giovanni Garbini, History and Ideology in Ancient Israel
Work type:
BOOK_REVIEW
Publication titles:
What Ship Goes There? 
Work type:
BOOK_CHAPTER
Publication titles:
William Robertson Smith, Lectures on the Religion of the Semites: Second and Third Series
Work type:
BOOK_REVIEW
Publication titles:
Marvin A. Sweeney, King Josiah of Judah: The Lost Messiah of Israel
Work type:
BOOK_REVIEW
Publication titles:
Eckart Otto, Das Deuteronomium: Politische Theologie und Rechtsreform in Juda und Assyrien
Work type:
BOOK_REVIEW
Publication titles:
George E. Mendenhall and Gary A. Herion, Ancient Israel’s Faith and History: An Introduction to the Bible in Context
Work type:
BOOK

TypeError: 'NoneType' object is not subscriptable

The ORCID Sandbox API uses  ORCID schema v2.0_rc2, substantially more granular than the current public API 
schema. v2.0_rc2 supports direct querying of these elements of the ORCID profile: 
activities, address, education, email, employment, external-identifiers, funding, keywords, 
researcher-urls, person, personal-details.  You must also supply credentials for OAUTH2 authentication.

In [91]:
SANDBOX_ORCID_API_BASE_URI = 'https://api.sandbox.orcid.org/v2.0_rc2/'

SANDBOX_ORCID_ID = '0000-0002-9323-699X/'

Possible sandbox API types, not including Lucene:

activities/
address/
education/
email/
employment/
external-identifiers/
funding/
keywords/
researcher-urls/
person/
personal-details/
other-names/

In [26]:
SANDBOX_API_TYPE = 'funding/'
S_URI = SANDBOX_ORCID_API_BASE_URI + SANDBOX_ORCID_ID + SANDBOX_API_TYPE


Guys, if you can get requests_oauthlib to function with ORCID sandbox authentication, more power to you.  I have no problem using the API sandbox with cURL and my long-lasting token:

curl -H "Accept: application/orcid+json" -H 'Authorization: Bearer [YOUR_LONG_LASTING_TOKEN]' 'https://api.sandbox.orcid.org/v2.0_rc2/0000-0002-9323-699X/activities/'

ORCID developers may use the v2.0_rc2 schema at their discretion:

In [92]:
URL = 'https://pub.orcid.org/v2.0_rc2/0000-0003-0902-4386/activities'

r = requests.get(URL, json=payload)
print(r.url)
r.json()

https://pub.orcid.org/v2.0_rc2/0000-0003-0902-4386/activities


{'educations': {'education-summary': [{'created-date': {'value': 1435248072718},
    'department-name': 'Computer Science',
    'end-date': None,
    'last-modified-date': {'value': 1442323001905},
    'organization': {'address': {'city': 'Milton Keynes',
      'country': 'GB',
      'region': 'Milton Keynes'},
     'disambiguated-organization': {'disambiguated-organization-identifier': '5488',
      'disambiguation-source': 'RINGGOLD'},
     'name': 'Open University'},
    'path': '/0000-0003-0902-4386/education/990546',
    'put-code': 990546,
    'role-title': '1st class with honours',
    'source': {'source-client-id': None,
     'source-name': {'value': 'Tom Demeranville'},
     'source-orcid': {'host': 'orcid.org',
      'path': '0000-0003-0902-4386',
      'uri': 'http://orcid.org/0000-0003-0902-4386'}},
    'start-date': None,
    'visibility': 'PUBLIC'}],
  'last-modified-date': {'value': 1442323001905}},
 'employments': {'employment-summary': [{'created-date': {'value': 14352

Next steps for the SHARE Curation Associates ORCID group:

Access points: do we begin by extracting ORCID IDs from SHARE db and create a loader for pushing selected ORCID data into SHARE?  Or do we try to cast a wider net, say, use an algorithm that attempts to match SHARE personal name strings + institutional affiliations + publications against ORCID data?  And just what data do we push from ORCID into SHARE?  External (researcher)IDs + Researcher IDs?  Affiliation history?  Funding history? That messy, messy publication data?